In [2]:
import os
from PIL import Image
import numpy as np

In [3]:
def read_image(im) :

    # Opening image as greyscale
    img = Image.open(im).convert('L')
    img.load()

    # Converting image into a numpy array
    data = np.asarray(img, dtype="int32")

    return data

In [4]:
trainFile = 'train_sample2.txt'
testFile = 'train_sample.txt'

In [5]:
labels = {}
images_labels = []
images = []
labelinv = {}

idx = 0

with open(trainFile, 'r') as f:
    for x in f:
        x = x.strip()
        if x:
            filePath, label = x.split(' ')
            if label not in labels:
                labels[label] = idx
                labelinv[idx] = label
                idx = idx + 1
            images_labels.append(labels[label])
            images.append(filePath)

In [21]:
def pca(images):
    A = np.zeros((len(images),128*128*3),dtype='float64')
    for idx, img in enumerate(images):
        im = Image.open(img).resize((128, 128), Image.ANTIALIAS)
        im = np.array(im).reshape(-1)
        A[idx,:] = im/255.0
    A_mean = np.mean(A, axis=0)
    A = A - A_mean
    
    covM = np.matmul(A, A.T)
    # can also use eigh as it is faster and returns the eigenvalues in sorted order (it works only for symmetric matrices, A x A^T is symmetric)
    eVals, eVecs = np.linalg.eig(covM)
    idx = eVals.argsort()[::-1]
    eVals = np.real(eVals[idx])
    eVecs = np.real(eVecs[:,idx])
    
    eVecs = np.matmul(A.T, eVecs)
    eVecsNorm = np.sqrt(np.sum(eVecs*eVecs, axis=0))
    actualEVecs = eVecs/eVecsNorm
    
    return (actualEVecs, A, A_mean)
# def PCA_decompose(images, num_comp):
    
#     data = None
#     MSE = np.zeros(60)
#     for x in images:
#         img = read_image(x)
#         img = img.flatten()
#         if data is None:
#             data = img
#         else:
#             data = np.vstack([data, img])

#     # Find mean of the images
#     mean = np.mean(data, axis=0)
    
#     # Center the matrix by subtracting the mean
#     centered = data - mean
    
#     # Find covariance matrix
#     cov = np.matmul(centered, centered.T)

#     # Retrieving the eigenvalues and eigenvectors
#     values, vectors = np.linalg.eig(cov)
#     idx = values.argsort()[::-1]
#     values = np.real(values[idx])
#     vectors = np.real(vectors[:,idx])
    
#     # Finding projection matrix
# #     projection = np.dot(vectors.T, centered)
    
#     eVecs = np.matmul(centered.T, vectors)
#     eVecsNorm = np.sqrt(np.sum(eVecs*eVecs, axis=0))
#     actualEVecs = np.real(eVecs/eVecsNorm)

#     return actualEVecs, np.real(centered), np.real(mean)

In [22]:
eVecs, A, toAdd = PCA_decompose(images, 32)
curEVecs = eVecs[:,0:32]
A_new = np.matmul(curEVecs.T, A.T).T

In [23]:
def softMax(x):
    # Subtract max(x) to compute the softmax of vector x in a numerically stable way
    shiftx = x - np.max(x)
    exps = np.exp(shiftx)
    return exps / np.sum(exps, axis = 0)

In [24]:
y = np.array(images_labels)
W = np.random.rand(len(labels), 32)

eta = 0.005
numIterations = 60000

for i in range(0, numIterations):
    wtx_labels = softMax(np.matmul(W, A_new.T))
    wtx_labels[y[:], np.arange(len(images))] += -1.0
    DSoftMax = (np.matmul(wtx_labels, A_new) / float(len(images))) + 0.0001 * W
    W = W - eta*(DSoftMax)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [25]:
correct = 0
total = 0

with open(testFile, 'r') as f:
    for x in f:
        x = x.strip()
        if x:
            filePath = x.split()[0]
#             im = Image.open(filePath).resize((255, 255), Image.ANTIALIAS)
            im = read_image(filePath)
            im = np.array(im).reshape(-1)
            im = im/255.0
            im = im
            im = np.matmul(curEVecs.T, im).T
        
            print(labelinv[np.argmax(softMax(np.matmul(W, im)))])
            total = total + 1
            if labelinv[np.argmax(softMax(np.matmul(W, im)))] == filePath.split('/')[-1].split('_')[0]:
                correct = correct + 1


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
006
